In [22]:
import pandas as pd

In [23]:
ivi2d = pd.read_excel('data/Internet Vacancy Index Feb 2025.xlsx', sheet_name='Seasonally Adjusted', dtype=str, keep_default_na=False)
ivi4d = pd.read_excel('data/Internet Vacancies Feb 2025.xlsx', sheet_name='4 digit 3 month average', dtype=str, keep_default_na=False)

anzsco = pd.read_csv('out/anzsco.csv', dtype=str)

/Users/xiao/pointlesswidgetsco/projects/ivi/.venv/lib/python3.13/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [24]:
val_vars = ['Level', 'ANZSCO_CODE', 'Title', 'State']
other_vars = ivi2d.columns.difference(val_vars)

ivi2d = ivi2d.melt(id_vars=val_vars, value_vars=other_vars, var_name='Date', value_name='Value')

# Convert the date column to date format
ivi2d['Date'] = pd.to_datetime(ivi2d['Date'])

ivi2d

,Level,ANZSCO_CODE,Title,State,Date,Value
0,1,0,AUSTRALIAN TOTAL,AUST,2006-01-01,212149.766
1,2,1,MANAGERS,AUST,2006-01-01,20471.54
2,3,11,"Chief Executives, Managing Directors & Legisla...",AUST,2006-01-01,502.97
3,3,12,Farmers and Farm Managers,AUST,2006-01-01,111.85
4,3,14,"Hospitality, Retail and Service Managers",AUST,2006-01-01,4298.75
...,...,...,...,...,...,...
117985,3,82,Construction and Mining Labourers,ACT,2025-02-01,14.317
117986,3,83,Factory Process Workers,ACT,2025-02-01,58.648
117987,3,84,"Farm, Forestry and Garden Workers",ACT,2025-02-01,10.586
117988,3,85,Food Preparation Assistants,ACT,2025-02-01,14.398


In [25]:
val_vars = ['ANZSCO_CODE', 'ANZSCO_TITLE', 'state']
other_vars = ivi4d.columns.difference(val_vars)

ivi4d = ivi4d.melt(id_vars=val_vars, value_vars=other_vars, var_name='Date', value_name='Value')

ivi4d = ivi4d.rename(columns={'ANZSCO_TITLE':'Title', 'state':'State'})

# Convert the date column to date format
ivi4d['Date'] = pd.to_datetime(ivi4d['Date'])

ivi4d

,ANZSCO_CODE,Title,State,Date,Value
0,0,Australia Total,AUST,2006-03-01,214668.935
1,1111,Chief Executives and Managing Directors,AUST,2006-03-01,158.85257
2,1112,General Managers,AUST,2006-03-01,308.89479
3,.,Legislators,AUST,2006-03-01,.
4,1211,Aquaculture Farmers,AUST,2006-03-01,0.66667
...,...,...,...,...,...
736663,8994,Motor Vehicle Parts and Accessories Fitters,ACT,2025-02-01,4
736664,8995,Printing Assistants and Table Workers,ACT,2025-02-01,0
736665,8996,Recycling and Rubbish Collectors,ACT,2025-02-01,0
736666,8997,Vending Machine Attendants,ACT,2025-02-01,0.66667


In [26]:
# Vacancies
vacs = {}

# Filter to keep only 1-digit numbers
vacs['1'] = ivi2d[ivi2d['ANZSCO_CODE'].astype(str).str.len() == 1]

# Filter to keep only 2-digit numbers
vacs['2'] = ivi2d[ivi2d['ANZSCO_CODE'].astype(str).str.len() == 2]

# Filter to keep only 4-digit numbers
vacs['4'] = ivi4d[ivi4d['ANZSCO_CODE'].astype(str).str.len() == 4]

In [27]:
# Aust/State aggregate - Trend analysis
vacancies_agg = vacs['1'][vacs['1']['ANZSCO_CODE']=='0'][['State','Date','Value']]
vacancies_agg = vacancies_agg.sort_values(['State','Date'])
vacancies_agg

,State,Date,Value
456,ACT,2006-01-01,3135.901
969,ACT,2006-02-01,3327.193
1482,ACT,2006-03-01,3496.769
1995,ACT,2006-04-01,3494.771
2508,ACT,2006-05-01,3672.584
...,...,...,...
115710,WA,2024-10-01,27760.25
116223,WA,2024-11-01,26450.952
116736,WA,2024-12-01,25999.967
117249,WA,2025-01-01,26931.78


In [28]:
vacancies_agg.to_csv('out/vacancies_agg.csv', index=False)

In [29]:
anzsco['Code'] = anzsco['Code'].str[:4]

anzsco_ = anzsco[['Code','Unit group', 'Minor group', 'Submajor group', 'Major group']].drop_duplicates()

In [30]:
vacancies_unitgroup = (
    vacs['4']
    .merge(anzsco_, left_on='ANZSCO_CODE', right_on='Code', how='left')
    [['State','Date','Code','Value','Unit group','Minor group','Submajor group','Major group']]
)

In [31]:
vacancies_unitgroup.to_csv('out/vacancies_unitgroup.csv', index=False)